# Requisitos

## Instalación de librerias necesarias

In [ ]:
%pip install boto3
%pip install sqlalchemy
%pip install PyMySQL

Python interpreter will be restarted.
Requirement already satisfied: boto3 in /databricks/python3/lib/python3.8/site-packages (1.16.7)
Requirement already satisfied: jmespath<1.0.0,>=0.7.1 in /databricks/python3/lib/python3.8/site-packages (from boto3) (0.10.0)
Requirement already satisfied: botocore<1.20.0,>=1.19.7 in /databricks/python3/lib/python3.8/site-packages (from boto3) (1.19.7)
Requirement already satisfied: s3transfer<0.4.0,>=0.3.0 in /databricks/python3/lib/python3.8/site-packages (from boto3) (0.3.7)
Requirement already satisfied: python-dateutil<3.0.0,>=2.1 in /databricks/python3/lib/python3.8/site-packages (from botocore<1.20.0,>=1.19.7->boto3) (2.8.1)
Requirement already satisfied: urllib3<1.26,>=1.25.4 in /databricks/python3/lib/python3.8/site-packages (from botocore<1.20.0,>=1.19.7->boto3) (1.25.11)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil<3.0.0,>=2.1->botocore<1.20.0,>=1.19.7->boto3) (1.15.0)
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: sqlalchemy in /local_disk0/.ephemeral_nfs/envs/pythonEnv-39b97941-302f-422c-bb3b-24f80ea3833e/lib/python3.8/site-packages (1.4.39)
Requirement already satisfied: greenlet!=0.4.17 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-39b97941-302f-422c-bb3b-24f80ea3833e/lib/python3.8/site-packages (from sqlalchemy) (1.1.2)
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: PyMySQL in /local_disk0/.ephemeral_nfs/envs/pythonEnv-39b97941-302f-422c-bb3b-24f80ea3833e/lib/python3.8/site-packages (1.0.2)
Python interpreter will be restarted.

## Importación de librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import boto3
import zipfile
import os
import io

# Ingesta de archivos de actualización

In [ ]:
session = boto3.Session(
aws_access_key_id=<access_key_id>,
aws_secret_access_key=<secret_access_key>
)

#Creating S3 Resource From the Session.
s3 = session.resource('s3')

data = 'deltas.zip'

s3.Bucket('proyecto-grupal-henry').download_file(data, 'deltas.zip')

with zipfile.ZipFile('deltas.zip', 'r') as zipF:
    zipF.extractall()

In [ ]:
customers = pd.read_csv('./delta_customers.csv')
order_items = pd.read_csv('./delta_order_items.csv', parse_dates=['shipping_limit_date'], infer_datetime_format=True)
order_payments = pd.read_csv('./delta_order_payments.csv')
order_reviews = pd.read_csv('./delta_order_reviews.csv', parse_dates=['creation_date','answer_timestamp'], infer_datetime_format=True)
orders = pd.read_csv('./delta_orders.csv', parse_dates=['purchase_timestamp','order_approved_at','delivered_carrier_date','delivered_customer_date','estimated_delivery_date'], infer_datetime_format=True)
products = pd.read_csv('./delta_products.csv')
sellers = pd.read_csv('./delta_sellers.csv')

# Limpieza y normalización

## Eliminación de columnas innecesarias

In [ ]:
customers = customers.drop('customer_unique_id', axis=1)

## Eliminación de duplicados

In [ ]:
order_reviews = order_reviews.drop_duplicates(subset='review_id', keep='first')

## Renombrar columnas

In [ ]:
customers = customers.rename(columns={'customer_zip_code_prefix':'zip_code_prefix', 'customer_city':'city', 'customer_state':'state'})

order_reviews = order_reviews.rename(columns={'review_comment_message':'comment_message', 'review_creation_date':'creation_date', 'review_comment_title':'comment_title', 'review_answer_timestamp':'answer_timestamp'})

order_items = order_items.rename(columns={'order_item_id':'product_quantity'})

sellers = sellers.rename(columns={'seller_zip_code_prefix':'zip_code_prefix', 'seller_city':'city', 'seller_state':'state'})

orders = orders.rename(columns={'order_purchase_timestamp':'purchase_timestamp', 'order_delivered_carrier_date':'delivered_carrier_date', 'order_delivered_customer_date':'delivered_customer_date', 'order_estimated_delivery_date':'estimated_delivery_date'})

products = products.rename(columns={'product_description_lenght':'description_length', 'product_name_lenght':'name_length', 'product_weight_g':'weight_g', 'product_length_cm':'length_cm', 'product_height_cm':'height_cm', 'product_width_cm':'width_cm', 'product_photos_qty':'photos_quantity'})

In [ ]:
order_payments.head()

Out[7]:

,payment_id,order_id,payment_type,payment_installments,payment_value
0,103887,be4knue39p6k2yhysoyo0i6qjqhsnvi6,credit_card,2,72.90
1,103888,6u2jh6bjietfdiqphvxexti575ng7dsf,boleto,1,13.58
2,103889,l0ojbi4m57vf7557akaeicnaz4vryiny,credit_card,3,146.94
3,103890,qelwitk82gph5z265qqus2el57p9l9lo,boleto,1,122.48
4,103891,bwmsru76jl7cg9qtp0jzzj5kjc4a1laf,voucher,5,698.11


## Completar valores nulos

In [ ]:
order_reviews['comment_message'].fillna('sem_comentarios',inplace=True)
order_reviews['comment_title'].fillna('sem_titulo',inplace=True)

# Actualización de la base de datos de MySQL en Amazon RDS

In [ ]:
# Creación de la conexión con la base de datos en Amazon RDS
engine = create_engine(<connection_details>)

In [ ]:
engine.execute("SET FOREIGN_KEY_CHECKS = 0")

Out[10]: <sqlalchemy.engine.cursor.LegacyCursorResult at 0x7f3e0015fc70>

In [ ]:
orders.to_sql('orders', con=engine, index=False, index_label='order_id', if_exists='append')

In [ ]:
customers.to_sql('customers', index=False, index_label='customer_id', con=engine, if_exists='append')

In [ ]:
order_reviews.to_sql('order_reviews', con= engine, index=False,index_label='review_id', if_exists='append')

In [ ]:
order_payments.to_sql('order_payments', engine, index=False, if_exists='append')

In [ ]:
order_items.to_sql('order_items', engine, index=False, if_exists='append')